# 🧠 M3c‴: Optimized HDC Semantic Transfer — Teacher

## Incorporates learnings from M2.5c + M2.5e

| Improvement | Source | Description |
|-------------|--------|-------------|
| HDC K-means curation | M2.5c | Cluster-based selection beats confidence threshold |
| Difficulty scoring | M2.5e | Distance from centroid = difficulty |
| Sharp curriculum data | M2.5e | Provide easy + hard examples for Student |

**GPU:** T4 is sufficient

---

In [ ]:
!pip install -q transformers datasets accelerate
!pip install -q firebase-admin
!pip install -q sentence-transformers
!pip install -q scikit-learn
print("✅ Dependencies installed")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import json
import numpy as np
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import base64

import firebase_admin
from firebase_admin import credentials, db

from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)
from sentence_transformers import SentenceTransformer

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🧠 M3c‴ Teacher — HDC K-means + Curriculum")

In [ ]:
# ============================================================
# FIREBASE CREDENTIALS - INSERT YOUR KEY HERE
# ============================================================
FIREBASE_CREDENTIALS = "INSERT KEY HERE"

FIREBASE_DATABASE_URL = "https://resonance-m3-default-rtdb.europe-west1.firebasedatabase.app"

if not firebase_admin._apps:
    cred = credentials.Certificate(FIREBASE_CREDENTIALS)
    firebase_admin.initialize_app(cred, {'databaseURL': FIREBASE_DATABASE_URL})
print("✅ Firebase initialized")

## HDC Encoder with Learned Projection

In [ ]:
class LearnedHDCEncoder(nn.Module):
    """
    HDC Encoder with learned projection.
    Trained with contrastive loss to preserve semantic similarity.
    """
    
    def __init__(self, hdc_dim=10000, semantic_dim=384):
        super().__init__()
        self.hdc_dim = hdc_dim
        self.semantic_dim = semantic_dim
        
        self.semantic_encoder = SentenceTransformer('all-MiniLM-L6-v2')
        
        self.projection = nn.Sequential(
            nn.Linear(semantic_dim, hdc_dim),
            nn.Tanh(),
        )
        nn.init.xavier_uniform_(self.projection[0].weight)
        
        print(f"✅ HDC Encoder: {semantic_dim}d → {hdc_dim}d")
    
    def get_semantic_embeddings(self, texts):
        if isinstance(texts, str):
            texts = [texts]
        with torch.no_grad():
            return self.semantic_encoder.encode(
                texts, convert_to_tensor=True, show_progress_bar=False
            )
    
    def forward(self, semantic_embeddings):
        return self.projection(semantic_embeddings)
    
    def encode(self, texts, quantize=True):
        semantic = self.get_semantic_embeddings(texts)
        if semantic.device != next(self.projection.parameters()).device:
            semantic = semantic.to(next(self.projection.parameters()).device)
        
        with torch.no_grad():
            hdc_vecs = self.forward(semantic)
        
        if quantize:
            hdc_np = hdc_vecs.cpu().numpy()
            threshold = 0.33
            return np.where(hdc_np > threshold, 1,
                           np.where(hdc_np < -threshold, -1, 0)).astype(np.int8)
        return hdc_vecs.cpu().numpy()
    
    def encode_batch(self, texts, batch_size=64, quantize=True):
        """Encode large list of texts in batches."""
        all_vecs = []
        for i in tqdm(range(0, len(texts), batch_size), desc="Encoding HDC"):
            batch = list(texts[i:i+batch_size])
            vecs = self.encode(batch, quantize=quantize)
            all_vecs.append(vecs)
        return np.vstack(all_vecs)

HDC_DIM = 10000
hdc_encoder = LearnedHDCEncoder(hdc_dim=HDC_DIM).to(device)

In [ ]:
def train_hdc_projection(encoder, texts, labels, epochs=10, batch_size=64):
    """Train projection with contrastive loss."""
    print("\n🔧 Training HDC Projection (contrastive)...")
    
    # Get semantic embeddings
    all_semantics = []
    for i in tqdm(range(0, len(texts), batch_size), desc="   Semantic encoding"):
        batch = list(texts[i:i+batch_size])
        emb = encoder.get_semantic_embeddings(batch)
        all_semantics.append(emb)
    all_semantics = torch.cat(all_semantics, dim=0).to(device)
    all_labels = torch.tensor(labels).to(device)
    
    optimizer = optim.Adam(encoder.projection.parameters(), lr=1e-3)
    encoder.projection.train()
    
    for epoch in range(epochs):
        total_loss = 0
        n_batches = 0
        perm = torch.randperm(len(all_semantics))
        
        for i in range(0, len(all_semantics), batch_size):
            idx = perm[i:i+batch_size]
            batch_sem = all_semantics[idx]
            batch_labels = all_labels[idx]
            
            optimizer.zero_grad()
            hdc_vecs = encoder.projection(batch_sem)
            hdc_norm = F.normalize(hdc_vecs, p=2, dim=1)
            
            sim_matrix = hdc_norm @ hdc_norm.T
            label_mask = (batch_labels.unsqueeze(0) == batch_labels.unsqueeze(1)).float()
            
            margin = 0.5
            pos_loss = ((1 - sim_matrix) * label_mask).sum() / (label_mask.sum() + 1e-8)
            neg_loss = (F.relu(sim_matrix - margin) * (1 - label_mask)).sum() / ((1 - label_mask).sum() + 1e-8)
            
            loss = pos_loss + neg_loss
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            n_batches += 1
        
        if (epoch + 1) % 2 == 0:
            print(f"   Epoch {epoch+1}/{epochs} | Loss: {total_loss/n_batches:.4f}")
    
    encoder.projection.eval()
    print("✅ Projection trained!")
    return encoder

## Load Data and Train Teacher

In [ ]:
MODEL_NAME = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
).to(device)

print(f"✅ Teacher: {MODEL_NAME}")

In [ ]:
print("Loading SST-2...")
dataset = load_dataset("glue", "sst2")

TRAIN_SIZE = 3000
TEST_SIZE = 500

train_texts = dataset['train']['sentence'][:TRAIN_SIZE]
train_labels = dataset['train']['label'][:TRAIN_SIZE]
test_texts = dataset['validation']['sentence'][:TEST_SIZE]
test_labels = dataset['validation']['label'][:TEST_SIZE]

print(f"✅ Train: {len(train_texts)}, Test: {len(test_texts)}")

In [ ]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(
            list(texts), truncation=True, padding='max_length',
            max_length=max_length, return_tensors='pt'
        )
        self.labels = torch.tensor(list(labels))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
test_dataset = SentimentDataset(test_texts, test_labels, tokenizer)

In [ ]:
def get_predictions_with_confidence(model, texts, tokenizer, batch_size=32):
    model.eval()
    predictions, confidences = [], []
    for i in tqdm(range(0, len(texts), batch_size), desc="Predicting"):
        batch_texts = list(texts[i:i+batch_size])
        inputs = tokenizer(
            batch_texts, truncation=True, padding=True,
            max_length=128, return_tensors='pt'
        ).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = F.softmax(outputs.logits, dim=1)
            preds = torch.argmax(probs, dim=1)
            confs = probs.max(dim=1).values
            predictions.extend(preds.cpu().numpy())
            confidences.extend(confs.cpu().numpy())
    return np.array(predictions), np.array(confidences)

print("📊 Teacher BEFORE training...")
preds_before, _ = get_predictions_with_confidence(model, test_texts, tokenizer)
acc_before = accuracy_score(test_labels, preds_before)
print(f"✅ Accuracy BEFORE: {acc_before:.1%}")

In [ ]:
training_args = TrainingArguments(
    output_dir="./teacher_output",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    warmup_steps=100,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="no",
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model, args=training_args,
    train_dataset=train_dataset, eval_dataset=test_dataset
)

print("\n🏋️ Training Teacher...")
trainer.train()
print("✅ Done!")

In [ ]:
print("📊 Teacher AFTER training...")
preds_after, _ = get_predictions_with_confidence(model, test_texts, tokenizer)
acc_after = accuracy_score(test_labels, preds_after)
print(f"✅ Accuracy AFTER: {acc_after:.1%} (+{acc_after - acc_before:.1%})")

## Train HDC Projection

In [ ]:
hdc_encoder = train_hdc_projection(
    hdc_encoder, train_texts, train_labels, epochs=10, batch_size=64
)

## HDC K-means Curation (from M2.5c)

Instead of confidence threshold, use K-means clustering:
- Cluster all HDC vectors
- Select nearest to centroid (canonical examples)

In [ ]:
print("\n" + "="*60)
print("🎯 HDC K-MEANS CURATION (M2.5c method)")
print("="*60)

# Get teacher predictions (we only want correct ones)
train_preds, train_confs = get_predictions_with_confidence(model, train_texts, tokenizer)
correct_mask = (train_preds == np.array(train_labels))
print(f"\n✅ Correct predictions: {correct_mask.sum()}/{len(train_texts)}")

# Filter to correct predictions only
correct_indices = np.where(correct_mask)[0]
correct_texts = [train_texts[i] for i in correct_indices]
correct_labels = [train_labels[i] for i in correct_indices]

# Encode all correct texts to HDC
print("\n🔄 Encoding to HDC space...")
all_hdc = hdc_encoder.encode_batch(correct_texts, batch_size=64, quantize=False)  # Float for K-means
print(f"   HDC vectors shape: {all_hdc.shape}")

In [ ]:
# K-means clustering
N_CLUSTERS = 500  # Half will be easy, half will be hard

print(f"\n🔮 K-means clustering (k={N_CLUSTERS})...")
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(all_hdc)
print(f"✅ Clustering complete")

In [ ]:
# For each cluster, compute distance from centroid for all members
# This gives us difficulty scores (closer = easier, farther = harder)

print("\n📊 Computing difficulty scores...")

difficulties = np.zeros(len(all_hdc))
for i in range(len(all_hdc)):
    cluster_id = cluster_labels[i]
    centroid = kmeans.cluster_centers_[cluster_id]
    difficulties[i] = np.linalg.norm(all_hdc[i] - centroid)

print(f"   Difficulty range: {difficulties.min():.2f} - {difficulties.max():.2f}")
print(f"   Difficulty mean: {difficulties.mean():.2f}")

In [ ]:
# Select examples for curriculum (from M2.5e: sharp curriculum)
# Easy: nearest to centroids (low difficulty)
# Hard: farthest from centroids (high difficulty)

N_EASY = 500   # Centroid-near examples
N_HARD = 500   # Boundary examples

print(f"\n🎯 Selecting curriculum examples...")
print(f"   Easy (centroid-near): {N_EASY}")
print(f"   Hard (boundary): {N_HARD}")

# Sort by difficulty
sorted_indices = np.argsort(difficulties)

# Easy = lowest difficulty (first N_EASY)
easy_indices = sorted_indices[:N_EASY]

# Hard = highest difficulty (last N_HARD)
hard_indices = sorted_indices[-N_HARD:]

print(f"\n   Easy difficulty range: {difficulties[easy_indices].min():.2f} - {difficulties[easy_indices].max():.2f}")
print(f"   Hard difficulty range: {difficulties[hard_indices].min():.2f} - {difficulties[hard_indices].max():.2f}")

In [ ]:
# Create HDC knowledge packet with difficulty scores
print("\n📦 Creating HDC Knowledge Packet with curriculum data...")

def quantize_to_ternary(vec):
    threshold = 0.33
    return np.where(vec > threshold, 1, np.where(vec < -threshold, -1, 0)).astype(np.int8)

hdc_knowledge = []

# Add easy examples
for idx in tqdm(easy_indices, desc="Easy examples"):
    hdc_vec = quantize_to_ternary(all_hdc[idx])
    hdc_knowledge.append({
        'hdc_vector': hdc_vec.tolist(),
        'label': int(correct_labels[idx]),
        'difficulty': float(difficulties[idx]),
        'curriculum': 'easy'
    })

# Add hard examples
for idx in tqdm(hard_indices, desc="Hard examples"):
    hdc_vec = quantize_to_ternary(all_hdc[idx])
    hdc_knowledge.append({
        'hdc_vector': hdc_vec.tolist(),
        'label': int(correct_labels[idx]),
        'difficulty': float(difficulties[idx]),
        'curriculum': 'hard'
    })

print(f"\n✅ Total examples: {len(hdc_knowledge)}")
print(f"   Easy: {sum(1 for x in hdc_knowledge if x['curriculum'] == 'easy')}")
print(f"   Hard: {sum(1 for x in hdc_knowledge if x['curriculum'] == 'hard')}")

In [ ]:
# Check label balance
easy_labels = [x['label'] for x in hdc_knowledge if x['curriculum'] == 'easy']
hard_labels = [x['label'] for x in hdc_knowledge if x['curriculum'] == 'hard']

print(f"\n📊 Label balance:")
print(f"   Easy - Pos: {sum(easy_labels)}, Neg: {len(easy_labels) - sum(easy_labels)}")
print(f"   Hard - Pos: {sum(hard_labels)}, Neg: {len(hard_labels) - sum(hard_labels)}")

In [ ]:
# Compress for upload
def compress_hdc_packet(knowledge):
    compressed = []
    for ex in knowledge:
        vec = np.array(ex['hdc_vector'], dtype=np.int8)
        mapped = (vec + 1).astype(np.uint8)
        packed = np.zeros(len(vec) // 4, dtype=np.uint8)
        for j in range(4):
            packed |= (mapped[j::4] << (j * 2))
        compressed.append({
            'hdc_packed': base64.b64encode(packed.tobytes()).decode('ascii'),
            'label': ex['label'],
            'difficulty': ex['difficulty'],
            'curriculum': ex['curriculum']
        })
    return compressed

compressed_knowledge = compress_hdc_packet(hdc_knowledge)

raw_size = len(hdc_knowledge) * HDC_DIM
compressed_size = sum(len(ex['hdc_packed']) for ex in compressed_knowledge)

print(f"\n📦 Compression:")
print(f"   Raw: {raw_size / 1024:.1f} KB")
print(f"   Compressed: {compressed_size / 1024:.1f} KB")
print(f"   Ratio: {raw_size / compressed_size:.1f}×")

In [ ]:
# Save projection weights
projection_weights = {
    'weight': hdc_encoder.projection[0].weight.cpu().detach().numpy().tolist(),
    'bias': hdc_encoder.projection[0].bias.cpu().detach().numpy().tolist(),
}

## Upload to Firebase

In [ ]:
print("\n📤 Uploading to Firebase...")

knowledge_packet = {
    'experiment': 'M3c_triple_prime_v2',
    'version': 'curriculum_optimized',
    'description': 'HDC K-means curation + Sharp curriculum (M2.5c + M2.5e)',
    'teacher_model': MODEL_NAME,
    'teacher_accuracy_before': float(acc_before),
    'teacher_accuracy_after': float(acc_after),
    'teacher_improvement': float(acc_after - acc_before),
    'hdc_dim': HDC_DIM,
    'projection_type': 'learned_contrastive',
    'projection_weights': projection_weights,
    'curation_method': 'kmeans_clustering',
    'n_clusters': N_CLUSTERS,
    'num_examples': len(compressed_knowledge),
    'num_easy': N_EASY,
    'num_hard': N_HARD,
    'curriculum_strategy': 'sharp',
    'examples': compressed_knowledge,
    'timestamp': datetime.now().isoformat(),
    'insights_from': ['M2.5c (HDC curation)', 'M2.5e (curriculum learning)']
}

ref = db.reference('sep_m3c_hdc_v3/knowledge_packet')
ref.set(knowledge_packet)

ref = db.reference('sep_m3c_hdc_v3/status')
ref.set({
    'teacher_ready': True,
    'timestamp': datetime.now().isoformat(),
    'hdc_dim': HDC_DIM,
    'num_examples': len(compressed_knowledge),
    'curriculum': 'sharp (easy→hard)',
    'teacher_accuracy': float(acc_after)
})

print(f"\n✅ Uploaded!")
print(f"\n📋 Summary:")
print(f"   Teacher: {acc_before:.1%} → {acc_after:.1%}")
print(f"   HDC dim: {HDC_DIM}")
print(f"   Curation: K-means ({N_CLUSTERS} clusters)")
print(f"   Curriculum: Sharp (easy→hard)")
print(f"   Examples: {len(compressed_knowledge)} ({N_EASY} easy + {N_HARD} hard)")

In [ ]:
# Save results
results = {
    'phase': 'M3c_triple_prime_v2',
    'node': 'teacher',
    'model': MODEL_NAME,
    'accuracy_before': float(acc_before),
    'accuracy_after': float(acc_after),
    'hdc_dim': HDC_DIM,
    'curation': 'kmeans',
    'n_clusters': N_CLUSTERS,
    'curriculum': 'sharp',
    'n_easy': N_EASY,
    'n_hard': N_HARD,
    'total_examples': len(compressed_knowledge),
    'optimizations': [
        'M2.5c: HDC K-means curation',
        'M2.5e: Sharp curriculum (easy→hard)',
        'M2.5e: Difficulty = distance from centroid',
        'Learned projection (contrastive)'
    ],
    'timestamp': datetime.now().isoformat()
}

with open('m3c_v3_teacher_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\n" + "="*60)
print("🎓 TEACHER COMPLETE — Run Student (L4) now!")
print("="*60)

In [ ]:
from google.colab import files
files.download('m3c_v3_teacher_results.json')